In [1]:
# Import our dependencies
%matplotlib inline
import numpy as np
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import sklearn as skl
import pandas as pd
import tensorflow as tf
import h5py
import keras
from keras import metrics

In [2]:
df = pd.read_csv("creditcardfraud_normalised.csv")
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,class
0,0.935192,0.766490,0.881365,0.313023,0.763439,0.267669,0.266815,0.786444,0.475312,0.510600,...,0.561184,0.522992,0.663793,0.391253,0.585122,0.394557,0.418976,0.312697,0.005824,0.0
1,0.978542,0.770067,0.840298,0.271796,0.766120,0.262192,0.264875,0.786298,0.453981,0.505267,...,0.557840,0.480237,0.666938,0.336440,0.587290,0.446013,0.416345,0.313423,0.000105,0.0
2,0.935217,0.753118,0.868141,0.268766,0.762329,0.281122,0.270177,0.788042,0.410603,0.513018,...,0.565477,0.546030,0.678939,0.289354,0.559515,0.402727,0.415489,0.311911,0.014739,0.0
3,0.941878,0.765304,0.868484,0.213661,0.765647,0.275559,0.266803,0.789434,0.414999,0.507585,...,0.559734,0.510277,0.662607,0.223826,0.614245,0.389197,0.417669,0.314371,0.004807,0.0
4,0.938617,0.776520,0.864251,0.269796,0.762975,0.263984,0.268968,0.782484,0.490950,0.524303,...,0.561327,0.547271,0.663392,0.401270,0.566343,0.507497,0.420561,0.317490,0.002724,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109095,0.945267,0.775710,0.861755,0.219453,0.770077,0.260500,0.269232,0.786090,0.441859,0.499398,...,0.558917,0.479100,0.662796,0.311702,0.571914,0.421178,0.414496,0.315261,0.000175,0.0
109096,0.978500,0.767318,0.839645,0.300379,0.764532,0.260523,0.265456,0.785193,0.481659,0.506978,...,0.557382,0.474237,0.663860,0.314923,0.610668,0.350053,0.416725,0.313564,0.001880,0.0
109097,0.977709,0.762571,0.803680,0.208215,0.780589,0.292035,0.265439,0.791531,0.452854,0.507137,...,0.561027,0.487366,0.662048,0.529265,0.615186,0.600702,0.414028,0.313231,0.005021,0.0
109098,0.977142,0.771956,0.852106,0.359707,0.763077,0.259376,0.265026,0.786537,0.435031,0.526833,...,0.558350,0.474510,0.667902,0.441509,0.585849,0.377726,0.416127,0.313513,0.000470,0.0


In [3]:
# Remove class target from features data
y = df["class"].values
X = df.drop(columns="class").values

In [4]:
Labels = df["class"].values
Features = df.drop(columns="class").values

In [5]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [6]:
number_input_features = len(X_train[0])

In [7]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy', threshold=0.6),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR')
]

In [8]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=16, activation="tanh", input_dim=number_input_features))
nn_model.add(tf.keras.layers.Dense(units=32, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [9]:
# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                480       
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,057
Trainable params: 1,057
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=METRICS)


In [11]:
# Train the model
fit_model = nn_model.fit(X_train, y_train, epochs=100)

Epoch 1/100
2558/2558 [==============================] - 10s 3ms/step - loss: 0.0165 - tp: 17.0000 - fp: 31.0000 - tn: 81620.0000 - fn: 157.0000 - accuracy: 0.9980 - precision: 0.3542 - recall: 0.0977 - auc: 0.7550 - prc: 0.0898
Epoch 2/100
2558/2558 [==============================] - 8s 3ms/step - loss: 0.0066 - tp: 84.0000 - fp: 30.0000 - tn: 81621.0000 - fn: 90.0000 - accuracy: 0.9984 - precision: 0.7368 - recall: 0.4828 - auc: 0.9262 - prc: 0.6122
Epoch 3/100
2558/2558 [==============================] - 6s 2ms/step - loss: 0.0053 - tp: 101.0000 - fp: 34.0000 - tn: 81617.0000 - fn: 73.0000 - accuracy: 0.9987 - precision: 0.7481 - recall: 0.5805 - auc: 0.9353 - prc: 0.6174
Epoch 4/100
2558/2558 [==============================] - 7s 3ms/step - loss: 0.0047 - tp: 117.0000 - fp: 37.0000 - tn: 81614.0000 - fn: 57.0000 - accuracy: 0.9988 - precision: 0.7597 - recall: 0.6724 - auc: 0.9355 - prc: 0.6223
Epoch 5/100
2558/2558 [==============================] - 6s 2ms/step - loss: 0.0043 - tp

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
# Evaluate the model using the test data
baseline_results = nn_model.evaluate(X_test,y_test,verbose=2)

853/853 - 3s - loss: nan - tp: 51.0000 - fp: 3.0000 - tn: 27206.0000 - fn: 15.0000 - accuracy: 0.9993 - precision: 0.9444 - recall: 0.7727 - auc: 0.9162 - prc: 0.8255 - 3s/epoch - 3ms/step
